# Heterogeneity Image Translation Transformer

### Multiscale Residual Spatiotemporal Vision Transformer (MRSt-ViT | PixFormer)
***

In [3]:
from main import *

hete = Heterogeneity()


------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.1.0+cu121 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080
------------------------------------------------------------



In [4]:
hete.make_dataloaders()

------------------------------------------------------------
-------------- DATA LOADING AND PREPROCESSING --------------
Total: 1000 - Training: 750 | Validation: 100 | Testing: 150
                        ... done ...                        
------------------------------------------------------------


In [5]:
hete.trainer()

------------------------------------------------------------
---------------------- MODEL TRAINING ----------------------
Total number of trainable parameters: 23,417,472


***
# END